# JupyterLite `xeus-sqlite` (demo)

* Krzysztof Molenda

---

`Xeus-sqlite` to jądro Jupyter oparte na bibliotece Xeus, które pozwala wykonywać zapytania SQL dla bazy SQLite bezpośrednio w notatniku jako “język” kernela zamiast Pythona. W JupyterLite działa całkowicie po stronie przeglądarki jako WebAssembly (WASM), co umożliwia interaktywną pracę z SQLite bez serwera i instalacji po stronie użytkownika.

Wykorzystywany głównie w nauczaniu SQL i podstaw baz danych z wykorzystaniem SQLite w przeglądarce, bez instalacji lokalnej i bez dostępu do serwera, idealne dla kursów i dokumentacji interaktywnej.

---

Kernel `xsqlite` zapewnia funkcjonalność _magicznych instrukcji_ do operacji na bazach danych z poziomu wiersza poleceń i natywne wykonywanie kodu SQL na podłączonej bazie danych.

```{note} Wyjaśnienie - magiczne instrukcje
**Cell magic** to specjalne polecenie IPython, które poprzedza się podwójnym procentem (`%%`) na pierwszej linii i które zmienia sposób wykonania całej komórki w notatniku Jupyter/JupyterLab/JupyterLite. Działa na cały blok komórki, w przeciwieństwie do **line magic** z pojedynczym `%`, które dotyczy tylko jednej linii.
```

## Tworzenie bazy danych

Aby utworzyć pustą bazę danych SQLite, wykonaj polecenie _line magic_:

In [1]:
%CREATE example_db.db

## Tworzenie tabel i ich zapełnianie

Tabele tworzy się i wypełnia za pomocą standardowych poleceń języka SQL:

In [2]:
CREATE TABLE players (Name STRING, Class STRING, Level INTEGER, Hitpoints INTEGER)

In [3]:
INSERT INTO players (Name, Class, Level, Hitpoints) VALUES ("Martin Splitskull", "Warrior", 3, 40)

In [4]:
SELECT COUNT(*) as rowcount FROM players

rowcount
1


Z poziomu pojedynczej komórki kodu można wykonać tylko jedno polecenie:

In [5]:
INSERT INTO players (Name, Class, Level, Hitpoints) VALUES ("Sir Wolf", "Cleric", 2, 20);

-- Poniższe elementy nie zostaną wstawione
INSERT INTO players (Name, Class, Level, Hitpoints) VALUES ("Sylvain, The Grey", "Wizard", 1, 10);

In [6]:
SELECT Name, Level, Hitpoints FROM players;

Name,Level,Hitpoints
Martin Splitskull,3,40
Sir Wolf,2,20


In [7]:
INSERT INTO players (Name, Class, Level, Hitpoints) VALUES ("Sylvain, The Grey", "Wizard", 1, 10);

In [8]:
SELECT Name, Level, Hitpoints FROM players;

Name,Level,Hitpoints
Martin Splitskull,3,40
Sir Wolf,2,20
"Sylvain, The Grey",1,10


## Zapytania do bazy

Obsługiwany jest pełen zakres poleceń zapytań SQL, łącznie z operacjami agregacji:

In [9]:
SELECT SUM (Level) FROM players

SUM (Level)
6


Grupowanie działa również:

In [10]:
SELECT Level, SUM(Hitpoints) AS `Total Hitpoints`
FROM players
GROUP BY Level
ORDER BY `Total Hitpoints` DESC;

Level,Total Hitpoints
3,40
2,20
1,10


## Administrowanie bazą danych

Zdefiniowano kilka magicznych linii wspomagających administrację bazą danych

In [11]:
%TABLE_EXISTS players

The table players exists.

In [12]:
%TABLE_EXISTS npcs

The table npcs doesn't exist.

In [13]:
%GET_INFO

Magic header string: SQLite format 3
Page size bytes: 4096
File format write version: 1
File format read version: 1
Reserved space bytes: 0
Max embedded payload fraction 64
Min embedded payload fraction: 32
Leaf payload fraction: 32
File change counter: 2
Database size pages: 2
First freelist trunk page: 2
Total freelist trunk pages: 1
Schema cookie: 2
Schema format number: 4
Default page cache size bytes: 0
Largest B tree page number: 0
Database text encoding: 1
User version: 0
Incremental vaccum mode: 0
Application ID: 0
Version valid for: 2
SQLite version: 3049002


## Łączenie się z inną bazą danych

Utworzenie nowej bazy danych spowoduje połączenie jądra z nową instancją bazy danych (i odłączenie od poprzedniej).

Tworzymy nową bazę danych (`potato.db`) i wypełniamy ją tabelą, a tabelę danymi:

In [14]:
%CREATE potato.db 

In [15]:
CREATE TABLE potaters(production INTEGER)

In [16]:
INSERT INTO potaters (production) VALUES (7)

In [17]:
SELECT * FROM potaters

production
7


Dostęp do poprzedniej bazydanych został utracony:

In [18]:
SELECT Name, Level, Hitpoints FROM players;

Error: no such table: players

```{warning} WAŻNE
W danym momencie tylko jedna baza danych jest aktywna
```

Możemy połączyć jadro z poprzednią bazą magiczna dyrektywą `%LOAD`:

In [ ]:
%LOAD example_db.db

In [ ]:
SELECT Name, Level, Hitpoints FROM players;